<a href="https://colab.research.google.com/github/KeyboarderSon/TIL/blob/main/RNN/RNN_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index={}
    self.word2count={}
    self.index2word={0:"SOS", 1:"EOS"}
    self.n_words=2 # SOS , EOS

In [3]:
# main reference
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# https://github.com/deeplearningzerotoall/PyTorch/blob/master/RNN/6-seq2seq.py


import random
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#sequence : 문장
#token : 단어

raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

# Start Of Sentence, End Of Sentence
SOS_token = 0
EOS_token = 1


# class for vocabulary related information of data
class Vocab:
  def __init__(self):
    self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
    self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
    self.vocab_count = {}
    self.n_vocab = len(self.vocab2index)

  def add_vocab(self, sentence):
    for word in sentence.split(" "):
      if word not in self.vocab2index:
        self.vocab2index[word] = self.n_vocab
        self.vocab_count[word] = 1
        self.index2vocab[self.n_vocab] = word
        self.n_vocab += 1
      else:#if word in self.vocab2index
        self.vocab_count[word] += 1


# source text, target text 분리
# 데이터가 잘 학습되도록 준비하는 전처리과정
def preprocess(corpus, source_max_length, target_max_length):
  print("reading corpus...")
  pairs = []
  for line in corpus:
    pairs.append([s for s in line.strip().lower().split("\t")])
  print("Read {} sentence pairs".format(len(pairs)))
  

## 나동빈 seq2seq

#### 독일어를 영어로

spaCy 라이브러리 : 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

In [4]:
%%capture

!python -m spacy download en
!python -m spacy download de

In [5]:
import  spacy
#영어 및 독일어 토큰화
spacy_en= spacy.load('en')
spacy_de= spacy.load('de')

In [6]:
#간단하게 토큰화 기능 사용해보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


영어 및 독일어 토큰화 함수 정의

In [7]:
def tokenize_de(text):
  #토큰화한 뒤 각각 토큰을 리스트에 담은 뒤 reverse it
  return [token.text for token in spacy_de.tokenizer(text)][::-1]


def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

field 라이브러리를 이용해 각 문장에 대한 구체적인 전처리 내용을 명시<br>
SRC 독일어 -> (TRG) 영어

In [8]:
from torchtext.data import Field, BucketIterator
#lower : 소문자로 처리하도록
SRC=Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG=Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [9]:
from torchtext.datasets import Multi30k

#각 문장을 Field에 정의된 내용을 기반으로 토큰화
train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 611kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 174kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 166kB/s]


In [10]:
print(f"training dataset 크기 : {len(train_dataset.examples)}")
print(f"validation dataset 크기 : {len(valid_dataset.examples)}")
print(f"testing dataset 크기 : {len(test_dataset.examples)}")

training dataset 크기 : 29000
validation dataset 크기 : 1014
testing dataset 크기 : 1000


In [11]:
#index 30에 해당하는 하나의 문장을 출력해보자
#번역하려는 독일어 문장
print(vars(train_dataset.examples[30])['src'])
#번역 결과로서 얻고자 하는 영어 문장
print(vars(train_dataset.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


```Field``` 객체의 build_vocab 메서드를 이용해 **영독 단어 사전**을 생성<br>
최소 2번 이상 등장한 단어만을 선택

In [12]:
#이를 입력차원으로 사용한다. 차원 수를 줄이기 위해 2번 이상 등장한 단어로 제한함
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


```SRC.vocab.stoi``` 
<br>토큰[어휘] : index

In [13]:
print(TRG.vocab.stoi["abcabc"])#없는 단어 : idx 0
print(TRG.vocab.stoi[TRG.pad_token])
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


```Field```에서 ```fix_length``` option을 사용하지 않았을 때 비숫한 길이의 문장을 하나의 batch에 할당하여 padding을 최소화 시켜준다. batch 별로 input dimension이 동일해야하므로 이러한 기능이 필요하다.

In [14]:
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE=128

train_iterator, valid_iterator, test_iterator=BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)

In [19]:
# 첫번째 배치에 대해서만

for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f"첫 번째 배치 크기 : {src.shape}")

  #첫번째 배치에 있는 첫번째 문장의 토큰 정보 보기
  
  #단어 개수가 n이면 idx 0 ~ n-1까지 주어지고 

  #문장의 처음과 끝부분은 2, 3 즉 <sos>와 <eos>이며

  #그 이후부터는 해당 batch에 포함되어있는 단어의 개수가 제일 많은 문장과 길이를 맞추기 위해 1로 padding이 들어감
  
  for i in range(src.shape[0]):
    print(f"idx {i}: {src[i][0].item()}")
  
  break

#[단어 개수, batch size] 현재 batch에 포함되는 단어들이 단어 개수만큼의 token으로 이뤄질 수 있도록 

첫 번째 배치 크기 : torch.Size([35, 128])
idx 0: 2
idx 1: 4
idx 2: 420
idx 3: 1583
idx 4: 44
idx 5: 35
idx 6: 92
idx 7: 33
idx 8: 37
idx 9: 550
idx 10: 5
idx 11: 3
idx 12: 1
idx 13: 1
idx 14: 1
idx 15: 1
idx 16: 1
idx 17: 1
idx 18: 1
idx 19: 1
idx 20: 1
idx 21: 1
idx 22: 1
idx 23: 1
idx 24: 1
idx 25: 1
idx 26: 1
idx 27: 1
idx 28: 1
idx 29: 1
idx 30: 1
idx 31: 1
idx 32: 1
idx 33: 1
idx 34: 1


##Encoder 구조
SRC 문장 -> context vector<br>


LSTM의 return :  hidden state, cell state<br>

**Hyperparameter**<br>
* input_dim : 하나의 단어에 대한 one hot encoding 차원<br>
* embed_dim : 임베딩(embeding) 차원(찐 input!!)<br>
* hidden_state<br>
* n_layers : RNN 레이어의 갯수<br>
* dropout_ratio

In [16]:
import torch.nn as nn

class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    #embedding? one-hot encoding을 특정 차원의 임베딩으로 매핑하는 레이어
    self.embedding = nn.Embedding(input_dim, embed_dim)

    #LSTM 레이어
    self.hidden_dim=hidden_dim
    self.n_layers=n_layers
    self.rnn=nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
    #input이 embeding을 거쳐 들어가기에 embed_dim만 존재,
    #return은 cell state, hidden state

    #dropout
    self.dropout=nn.Dropout(dropout_ratio)

  #인코더는 소스 문장을 입력으로 받아 context vector을 반환
  def forward(self, src):
    
    embedded = self.dropout(self.embedding(src))
    # src[단어개수, batch size] -> embedded[단어개수, batch size, embedding dim]

    output, (hidden, cell) = self.rnn(embedded)
    #output[단어개수, batch size, hidden dim] : 현재 단어의 출력 정보
    #hidden[n_layer, batch size, hidden dim] : 현재까지의 모든 단어의 정보
    #cell[n_layer, batch size, hidden dim] : 현재 까지의 모든 단어의 정보

    #  *** hidden과 cell의 차이는 그럼 뭐지? ***

    # context vector 반환
    return hidden, cell


### Decoder

context vector을 타겟 문장으로 디코딩한다.
LSTM은 hidden state와 cell state를 반환한다.

Hyperparameter<br>
input dim<br>
embed dim<br>
hidden dim<br>
n layers<br>
dropout ratio

In [17]:
class Decoder(nn.Module):
  def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    self.embedding=nn.embedding(output_dim, embed_dim)
    